<a href="https://colab.research.google.com/github/kokochka/veggies-classifier/blob/main/veg_recognition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
!pip install tensorflow keras kaggle

In [7]:
import os
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json


cp: cannot stat 'kaggle.json': No such file or directory
chmod: cannot access '/root/.kaggle/kaggle.json': No such file or directory


In [6]:
from kaggle.api.kaggle_api_extended import KaggleApi

# Авторизация в Kaggle
api = KaggleApi()
api.authenticate()

# Скачивание датасета
dataset_path = 'vegetable-image-dataset'
api.dataset_download_files('misrakahmed/vegetable-image-dataset', path=dataset_path, unzip=True)


OSError: Could not find kaggle.json. Make sure it's located in /root/.kaggle. Or use the environment method.

In [4]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Путь к директории с изображениями
base_dir = '/content/vegetable-image-dataset/Vegetable Images'

# Размеры изображений
img_height, img_width = 150, 150

# Подготовка генераторов данных
datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)

train_generator = datagen.flow_from_directory(
    '/content/vegetable-image-dataset/Vegetable Images/train',
    target_size=(img_height, img_width),
    batch_size=32,
    class_mode='categorical',
    subset='training'
)

validation_generator = datagen.flow_from_directory(
    '/content/vegetable-image-dataset/Vegetable Images/validation',
    target_size=(img_height, img_width),
    batch_size=32,
    class_mode='categorical',
    subset='validation'
)


test_generator = datagen.flow_from_directory(
     '/content/vegetable-image-dataset/Vegetable Images/test',
     target_size=(img_height, img_width),
     batch_size=32,
     class_mode='categorical'
 )


FileNotFoundError: [Errno 2] No such file or directory: '/content/vegetable-image-dataset/Vegetable Images/train'

In [10]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(img_height, img_width, 3)),
    MaxPooling2D(pool_size=(2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Flatten(),
    Dense(512, activation='relu'),
    Dropout(0.5),
    Dense(15, activation='softmax')
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


In [11]:
checkpoint_cb = ModelCheckpoint(
    'vegetable_classifier_epoch_{epoch:02d}_val_acc_{val_accuracy:.2f}.h5',
    save_best_only=False,
    save_weights_only=False,
    save_freq='epoch'
)


In [ ]:
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // validation_generator.batch_size,
    epochs=35,
    callbacks=[checkpoint_cb]  # викликаємо збереження моделі після кожної епохи
)

# оцінка моделі протягом навчання
test_loss, test_acc = model.evaluate(validation_generator, verbose=2)
print(f'\nTest accuracy: {test_acc}')



Epoch 1/35
375/375 [==============================] - ETA: 0s - loss: 1.2898 - accuracy: 0.5777

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


375/375 [==============================] - 710s 2s/step - loss: 1.2898 - accuracy: 0.5777 - val_loss: 0.5768 - val_accuracy: 0.8125
Epoch 2/35
375/375 [==============================] - 700s 2s/step - loss: 0.5529 - accuracy: 0.8221 - val_loss: 0.3108 - val_accuracy: 0.9149
Epoch 3/35
375/375 [==============================] - 704s 2s/step - loss: 0.3381 - accuracy: 0.8891 - val_loss: 0.2607 - val_accuracy: 0.9062
Epoch 4/35
375/375 [==============================] - 703s 2s/step - loss: 0.2087 - accuracy: 0.9338 - val_loss: 0.1946 - val_accuracy: 0.9514
Epoch 5/35
375/375 [==============================] - 711s 2s/step - loss: 0.1625 - accuracy: 0.9487 - val_loss: 0.1677 - val_accuracy: 0.9531
Epoch 6/35
375/375 [==============================] - 693s 2s/step - loss: 0.1250 - accuracy: 0.9615 - val_loss: 0.1449 - val_accuracy: 0.9601
Epoch 7/35
375/375 [==============================] - 699s 2s/step - loss: 0.1056 - accuracy: 0.9689 - val_loss: 0.2683 - val_accuracy: 0.9288
Epoch 8/35

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [1]:
model.save('vegetable_classifier_model.h5')


NameError: name 'model' is not defined

In [8]:
import numpy as np
from tensorflow.keras.preprocessing import image

def predict_image(img_path):
    img = image.load_img(img_path, target_size=(img_height, img_width))
    img_array = image.img_to_array(img) / 255.0
    img_array = np.expand_dims(img_array, axis=0)

    prediction = model.predict(img_array)
    predicted_class = np.argmax(prediction, axis=1)
    class_labels = {v: k for k, v in train_generator.class_indices.items()}

    return class_labels[predicted_class[0]]

# Пример использования
img_path = '/content/cucumber.jpeg'  # Замените на путь к вашему изображению
print(predict_image(img_path))


NameError: name 'model' is not defined